# Warsztaty Python w Data Science

---
## Data Wrangling z Biblioteką Pandas 
### Część 3 z 3 - Tidy Data

[Hadley Wickham - **Tidy Data**](https://www.jstatsoft.org/index.php/jss/article/view/v059i10/v59i10.pdf)


[Wikipedia - Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)  



---

### Zadanie 2

Do poniższego DataFrame:
    
`df = pd.DataFrame(np.random.randint(1,100, 80).reshape(8, 10))`

Dołożyć:
    
- kolumnę z sumą po wierszach
- wiersz z sumą po kolumnach

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randint(1,100, 80).reshape(8, 10))

In [ ]:
df

In [ ]:
def sumy(df):
    df['Total'] = df.sum(axis=1)
    i, j = df.shape
    df.loc[i+1] = df.sum(axis=0)
    return df

sumy(df)

#### Zadanie 3

Przerobic DataFrame:

In [ ]:
data = {'weekday': ["Monday", "Tuesday", "Wednesday", 
         "Thursday", "Friday", "Saturday", "Sunday"],
        'Person 1': [12, 6, 5, 8, 11, 6, 4],
        'Person 2': [10, 6, 11, 5, 8, 9, 12],
        'Person 3': [8, 5, 7, 3, 7, 11, 15]}
df = pd.DataFrame(data, columns=['weekday',
        'Person 1', 'Person 2', 'Person 3'])

In [ ]:
df

Na postać "tidy" (1 wiersz na ocenę)

In [ ]:
melted = pd.melt(df, id_vars=["weekday"], 
                 var_name="Person", value_name="Score")
melted

#### Zadanie 4

Biorąc następujący DataFrame:

In [ ]:
df = pd.DataFrame(data = {
    'Day' : ['MON', 'TUE', 'WED', 'THU', 'FRI'], 
    'Google' : [1129,1132,1134,1152,1152], 
    'Apple' : [191,192,190,190,188] 
})
df

In [ ]:
reshaped_df = df.melt(id_vars=['Day'], var_name='Company', value_name='Closing Price')
reshaped_df

Stworzyć korzystając z `reshaped_df` przy uzyciu `pivot` DataFrame identyczny z `df`

In [ ]:
original_df = reshaped_df.pivot(index='Day', columns='Company')['Closing Price'].reset_index()
original_df.columns.name = None
original_df

---
## Grupowanie 

In [ ]:
import numpy as np

df = pd.DataFrame({'value': np.random.randint(0, 100, 20)})
df

In [ ]:
labels = ["{0} - {1}".format(i, i + 9) for i in range(0, 100, 10)]
labels

In [ ]:
pd.cut(df.value, range(0, 105, 10), right=False)

In [ ]:
pd.cut(df.value, range(0, 105, 10), right=False, labels=labels)

In [ ]:
df['Group'] = pd.cut(df.value, range(0, 105, 10), right=False, labels=labels)
df

In [ ]:
df.groupby('Group').count()

In [ ]:
df.groupby('Group').sum()

In [ ]:
df.groupby('Group').agg({'value': ['count', sum]})

In [ ]:
import functools

def test_sum(series):
       return functools.reduce(lambda x, y: x + y, series, 0)

test_sum(range(10))

In [ ]:
df.groupby('Group').agg({'value': ['count', test_sum]})

#### Zadanie 1

Napraw `df_e` biorąc maksimum wartości `baz` dla pary `foo`, `bar` (odrzuć konflikty)

In [ ]:
df_e = pd.DataFrame({"foo": ['one', 'one', 'two', 'two'],
                   "bar": ['A', 'A', 'B', 'C'],
                   "baz": [1, 2, 3, 4]})

In [ ]:
df_e

In [ ]:
df_e.groupby(['foo','bar']).aggregate('max')

In [ ]:
df_e.groupby(['foo','bar']).aggregate('max').reset_index()

---
## Tidy Data
### Pamiętajmy o podstawowych zasadach:  
1. Każda zmienna tworzy kolumnę.
2. Każda obserwacja stanowi wiersz.
3. Dane w jednej kolumnie są przechowywane w jednym formacie.

In [ ]:
url1 = 'https://s3.amazonaws.com/assets.datacamp.com/production/course_1273/datasets/df1.csv'
url2 = 'https://s3.amazonaws.com/assets.datacamp.com/production/course_1273/datasets/df2.csv'

df1 = pd.read_csv(url1, sep = ',')
df2 = pd.read_csv(url2, sep = ',')

#### Jakie zasady nie spełniają załadowane tabele?

In [ ]:
df1

In [ ]:
df2


df2 --> reguła 2  
Obserwujemy coś w jakimś momencie, dlatego czasowe dane nie mogą stanowić nagłówków.



#### Zrób dane bardziej 'tidy'

In [ ]:
df2_melted = pd.melt(df2, id_vars=['Country'])
df2_melted

#### Metoda 'split'

In [ ]:
df1.columns

In [ ]:
columns = [c if c != 'owner' else 'name' for c in df1.columns]
columns

In [ ]:
df1['surnames'] = ['Escobar', 'Potter', 'Connor']
df1

In [ ]:
df1['full name'] = df1.owner + ' ' + df1.surnames
df1 = df1.drop(['owner', 'surnames'], axis=1)
df1

In [ ]:
'Jason Escobar'.split()

In [ ]:
# error
df1['name'] = df1['full name'].split(' ')

In [ ]:
def f(x):
    return x.split()

In [ ]:
df1['name'] = df1['full name'].apply(f)
df1

## Lambda

In [ ]:
df1['name'] = df1['full name'].apply(lambda x: x.split(' '))
df1

In [ ]:
df1['name'] = df1['full name'].apply(lambda x: x.split(' ')[0])
df1

In [ ]:
df2_tidy = df2_melted.rename(columns = {'variable': 'Year', 'value': 'Income'})
df2_tidy

In [ ]:
df2_melted

In [ ]:
df2_melted.rename(columns = {'variable': 'Year', 'value': 'Income'}, inplace=True)
df2_melted

#### Formatowanie

In [ ]:
df2_melted.dtypes

In [ ]:
df2_melted['Year'] = df2_melted['Year'].apply(lambda x: x[1:5])
df2_melted

In [ ]:
df2_melted['Year'].apply(pd.to_numeric)

In [ ]:
df2_melted['Year'].astype('int64')
df2_melted

### Więcej zabawy z danymi

In [ ]:
messy = pd.DataFrame({'First' : ['John', 'Jane', 'Mary'], 
                      'Last' : ['Smith', 'Doe', 'Johnson'], 
                      'Treatment A' : [np.nan, 16, 3], 
                      'Treatment B' : [2, 11, 1]})
messy

In [ ]:
messy.transpose()

In [ ]:
messy.T

In [ ]:
tidy = pd.melt(messy, 
               id_vars=['First','Last'], 
               var_name='treatment', 
               value_name='result')
tidy

In [ ]:
tidy['Name'] = tidy['First'] + ' ' + tidy['Last']

In [ ]:
messy1 = tidy.pivot(index='Name',columns='treatment',values='result')
messy1

In [ ]:
messy1.index

In [ ]:
messy1.reset_index(inplace=True)
messy1

#### Braki danych

In [ ]:
missing_f = pd.DataFrame([[1,1,1,1,2,2,2,2],[1,np.nan,np.nan,np.nan,2,np.nan,np.nan,np.nan]])
missing_b = pd.DataFrame([[1,1,1,1,2,2,2,2],[np.nan,np.nan,np.nan,1,np.nan,np.nan,np.nan,2]])
display(missing_f)

In [ ]:
missing_f.T

In [ ]:
missing_f.T.fillna(method='ffill')

##### Uzupełnić braki w missing_b metodą 'bfill'

In [ ]:
missing_b.T

In [ ]:
missing_b.T.fillna(method='bfill')

---

In [ ]:
pd.read_excel('data/Matma1.xlsx', index_col=0)

`conda install xlrd`

#### Zadanie 1. Przedstaw sprawozdanie z ankiety studenckiej
- Wyczyść w/w dane (doprowadź do postaci Tidy; prawdopodobnie rozbijając na kilka zbiorów danych)
- Przedstaw wnioski z ankiety - "Dlaczego wybrałeś studia w SGH?", który wykład był najciekawszy itd. itd.